In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import tensorflow as tf
import tqdm
import io
import imageio

from collections import deque
from skimage.transform import resize

# Own libraries and modules
from helpers import loading, plotting, utils, summaries
from models import compression

In [ ]:
training = {
    'n_epochs': 50000,
    'batch_size': 32,
    'patch_size': 128
}

In [ ]:
patch_size = 128
n_latent = 256
n_latent_bytes = 1/8

bitmap_size = patch_size*patch_size*3
compression_rate = bitmap_size / (n_latent_bytes * n_latent)
compression_bpp = 8 * n_latent * n_latent_bytes / patch_size / patch_size

print('Bitmap size: {:,d} bytes'.format(bitmap_size))
print('Latent size: {:,}-D'.format(n_latent))
print('Latent repr: {:,} bytes'.format(n_latent * n_latent_bytes))
print('Compression rate: 1:{}'.format(compression_rate))
print('Compression Fi  : {:.2f} bpp'.format(compression_bpp))

## Load the dataset

In [ ]:
class IPDataset:
    
    def __init__(self, data_directory, randomize=False, load='xy', val_patch_size=128, val_n_patches=2, n_images=120, v_images=30):
        self.files = {}
        self.files['training'], self.files['validation'] = loading.discover_files(data_directory, randomize=randomize, n_images=n_images, v_images=v_images)
                
        self.data = {
            'training': loading.load_images(self.files['training'], data_directory=data_directory, load=load),
            'validation': loading.load_patches(self.files['validation'], data_directory=data_directory, patch_size=val_patch_size // 2, n_patches=val_n_patches, load=load, discard_flat=True)
        }
        
        if 'x' in self.data['training']:
            self.H, self.W = self.data['training']['x'].shape[1:3]
        else:
            self.H, self.W = self.data['training']['y'].shape[1:3]
            
        print('Loaded dataset with {}x{} images'.format(self.W, self.H))
        
    def __getitem__(self, key):
        if key in ['training', 'validation']:
            return self.data[key]
        else:
            return super().__getitem__(key)
        
    def next_training_batch(self, batch_id, batch_size, patch_size, discard_flat=True):
        batch_x = np.zeros((batch_size, patch_size, patch_size, 3), dtype=np.float32)
        for b in range(batch_size):
            
            found = False            
            panic_counter = 5
            
            while not found:
                xx = np.random.randint(0, self.W - patch_size)
                yy = np.random.randint(0, self.H - patch_size)

                patch = self.data['training']['y'][batch_id * batch_size + b, yy:yy + patch_size, xx:xx + patch_size, :].astype(np.float) / (2**8 - 1)

                # Check if the found patch is acceptable: eliminate empty patches
                if discard_flat:
                    patch_variance = np.var(patch)
                    if patch_variance < 1e-2:
                        panic_counter -= 1
                        found = False if panic_counter > 0 else True
                    elif patch_variance < 0.02:
                        found = np.random.uniform() > 0.5
                    else:
                        found = True
                else:
                    found = True
            
            batch_x[b, :, :, :] = patch
        return batch_x

    def next_validation_batch(self, batch_id, batch_size):
        patch_size = self.data['validation']['y'].shape[1]
        batch_x = np.zeros((batch_size, patch_size, patch_size, 3), dtype=np.float32)
        for b in range(batch_size):
            batch_x[b, :, :, :] = self.data['validation']['y'][batch_id * batch_size + b].astype(np.float)
        return batch_x

In [ ]:
# Load data
# camera_name = "Nikon D90"
# data_directory = os.path.join('./data/raw/nip_training_data/', camera_name)
data_directory = os.path.join('./data/raw/raise/')

data = IPDataset(data_directory, n_images=7850, v_images=250, load='y')

for dataset in ['training', 'validation']:
    print('{} : {}'.format(dataset, data[dataset]['y'].shape))

## Define the Deep Compression Network Models

In [ ]:
class AutoencoderDCN(compression.DCN):
    
    def construct_model(self, n_filters=8, n_fscale=2, n_latent=1024, kernel=5, n_layers=3, dropout=0.0):
        
        self.n_layers = n_layers
        self.n_latent = n_latent
        self.n_filters = n_filters
        self.n_fscale = n_fscale
        self.kernel = kernel        
        
        activation = tf.nn.leaky_relu
        latent_activation = None
        last_activation = None

        print('Building Deep Compression Network with d-latent={}'.format(self.n_latent))

        net = self.x
        print('in size: {}'.format(net.shape))

        # Convolutions
        n_filters = self.n_filters

        for r in range(self.n_layers):
            act = activation if (n_latent > 0 or (n_latent == 0 and r < self.n_layers - 1)) else latent_activation
            net = tf.contrib.layers.conv2d(net, n_filters, self.kernel, stride=2, 
                                           activation_fn=activation if (n_latent > 0 or (n_latent == 0 and r < self.n_layers - 1)) else latent_activation, 
                                           scope='dcn{}/encoder/conv_{}'.format(self.label, r))
        #     print('net size: {}'.format(net.shape))
#                     net = tf.contrib.layers.max_pool2d(net, 2, scope='dcn{}/pool_{}'.format(self.label, r))
            print('cnn size: {} + {}'.format(net.shape, act))
            n_filters *= self.n_fscale

        # Flatten and get latent representation
        
        if n_latent > 0:
            
            flat = tf.contrib.layers.flatten(net, scope='dcn{}/encoder/flatten_{}'.format(self.label, 0))
            print('net size: {}'.format(flat.shape))

            flat = tf.contrib.layers.fully_connected(flat, self.n_latent, activation_fn=latent_activation, scope='dcn{}/encoder/dense_{}'.format(self.label, 0))
            latent = tf.identity(flat, name='dcn{}/latent'.format(self.label))
            print('latent size: {} + {}'.format(latent.shape, latent_activation))

            inet = tf.contrib.layers.fully_connected(latent, int(flat.shape[-1]), activation_fn=activation, scope='dcn{}/decoder/dense_{}'.format(self.label, 0))
            print('net size: {} + {}'.format(inet.shape, activation))
            
            self.uses_bottleneck = True
            
        else:
            flat = tf.contrib.layers.flatten(net, scope='dcn{}/encoder/flat_{}'.format(self.label, 0))
            latent = tf.identity(flat, name='dcn{}/latent'.format(self.label))
            print('latent size: {}'.format(latent.shape))
            self.n_latent = int(np.prod(latent.shape[1:]))
            self.uses_bottleneck = False
            inet = latent
            
        inet = tf.reshape(inet, tf.shape(net, name='dcn{}/decoder/shape'.format(self.label)), name='dcn{}/decoder/reshape_{}'.format(self.label, 0))
        print('net size: {}'.format(inet.shape))

        # Transposed convolutions
        for r in range(self.n_layers):
            n_filters = n_filters // self.n_fscale
            inet = tf.contrib.layers.conv2d(inet, 2 * n_filters, self.kernel, stride=1, 
                                            activation_fn=last_activation if r == self.n_layers - 1 else activation, 
                                            scope='dcn{}/decoder/tconv_{}'.format(self.label, r))
            print('cnn size: {} + {}'.format(inet.shape, last_activation if r == self.n_layers - 1 else activation))
            inet = tf.depth_to_space(inet, 2, name='dcn{}/decoder/d2s_{}'.format(self.label, r))
#             inet = tf.contrib.layers.conv2d_transpose(inet, 3 if r == self.n_layers - 1 else n_filters, self.kernel, stride=2, 
#                                                         activation_fn=last_activation if r == self.n_layers - 1 else activation,
#                                                         scope='dcn{}/tconv_{}'.format(self.label, r))
            print('d2s size: {} + {}'.format(inet.shape, None))
            

        inet = tf.contrib.layers.conv2d(inet, 3, 1, stride=1, activation_fn=last_activation, scope='dcn{}/decoder/tconv_out'.format(self.label))
        print('out size: {} + {}'.format(inet.shape, last_activation))

        y = tf.identity(inet, name='y')

        with tf.name_scope('dcn{}/optimization'.format(self.label)):
            lr = tf.placeholder(tf.float32, name='dcn_learning_rate')
#             loss = tf.abs(self.x - y) 
            loss = tf.nn.l2_loss(self.x - y)
            adam = tf.train.AdamOptimizer(learning_rate=lr)
            opt = adam.minimize(loss, var_list=self.parameters)
            
        return y, lr, loss, adam, opt, latent
    
    def short_name(self):
        return '{}-C{}{}'.format(super().short_name(), self.n_layers, 'D' if self.uses_bottleneck else '')

In [ ]:
class TwitterDCN(compression.DCN):
    """
    Auto-encoder architecture described in:
    [1] L. Theis, W. Shi, A. Cunningham, and F. Huszár, “Lossy Image Compression with Compressive Autoencoders,” Mar. 2017.
    """
    
    def construct_model(self):
        
        activation = tf.nn.leaky_relu
        latent_activation = tf.nn.tanh
        last_activation = tf.nn.sigmoid
        
        self.n_layers = 9
        self.n_latent = 

        print('Building Deep Compression Network with d-latent={}'.format(self.n_latent))
        
        
        with tf.name_scope('dcn{}/normalization'.format(self.label)):
            net = 2 * (self.x - 0.5)
            print('net size: {}'.format(net.shape))

        # Convolutions        
#         n_filters = self.n_filters
        
        net = tf.contrib.layers.conv2d(net,  64, 5, stride=2, activation_fn=activation, scope='dcn{}/encoder/conv_{}'.format(self.label, 0))
        net = tf.contrib.layers.conv2d(net, 128, 5, stride=2, activation_fn=None, scope='dcn{}/encoder/conv_{}'.format(self.label, 1))
        
        resnet = tf.contrib.layers.conv2d(tf.nn.leaky_relu(net, name='dcn{}/encoder/conv_{}/lrelu'.format(self.label, 1)), 128, 3, stride=1, activation_fn=activation, scope='dcn{}/encoder/conv_{}'.format(self.label, 2))
        resnet = tf.contrib.layers.conv2d(resnet, 128, 3, stride=1, activation_fn=None, scope='dcn{}/encoder/conv_{}'.format(self.label, 3))
        net = tf.add(net, resnet, name='dcn{}/encoder/sum_a{}'.format(self.label, 0))

        resnet = tf.contrib.layers.conv2d(net, 128, 3, stride=1, activation_fn=activation, scope='dcn{}/encoder/conv_{}'.format(self.label, 4))
        resnet = tf.contrib.layers.conv2d(resnet, 128, 3, stride=1, activation_fn=None, scope='dcn{}/encoder/conv_{}'.format(self.label, 5))
        net = tf.add(net, resnet, name='dcn{}/encoder/sum_b{}'.format(self.label, 1))

        resnet = tf.contrib.layers.conv2d(net, 128, 3, stride=1, activation_fn=activation, scope='dcn{}/encoder/conv_{}'.format(self.label, 6))
        resnet = tf.contrib.layers.conv2d(resnet, 128, 3, stride=1, activation_fn=None, scope='dcn{}/encoder/conv_{}'.format(self.label, 7))
        net = tf.add(net, resnet, name='dcn{}/encoder/sum_c{}'.format(self.label, 2))
        
        net = tf.contrib.layers.conv2d(net, 96, 5, stride=2, activation_fn=None, scope='dcn{}/encoder/conv_{}'.format(self.label, 8))
        
        latent = tf.identity(net, name='dcn{}/latent'.format(self.label))
                
        inet = tf.contrib.layers.conv2d(latent, 512, 3, stride=1, activation_fn=None, scope='dcn{}/decoder/conv_{}'.format(self.label, 0))
        inet = tf.depth_to_space(inet, 2, name='dcn{}/decoder/d2s_{}'.format(self.label, 0))
        
        resnet = tf.contrib.layers.conv2d(inet, 128, 3, stride=1, activation_fn=activation, scope='dcn{}/decoder/conv_{}'.format(self.label, 1))
        resnet = tf.contrib.layers.conv2d(resnet, 128, 3, stride=1, activation_fn=None, scope='dcn{}/decoder/conv_{}'.format(self.label, 2))
        inet = tf.add(inet, resnet, name='dcn{}/decoder/sum_a{}'.format(self.label, 0))
        
        resnet = tf.contrib.layers.conv2d(inet, 128, 3, stride=1, activation_fn=activation, scope='dcn{}/decoder/conv_{}'.format(self.label, 3))
        resnet = tf.contrib.layers.conv2d(resnet, 128, 3, stride=1, activation_fn=None, scope='dcn{}/decoder/conv_{}'.format(self.label, 4))
        inet = tf.add(inet, resnet, name='dcn{}/decoder/sum_b{}'.format(self.label, 1))
        
        resnet = tf.contrib.layers.conv2d(inet, 128, 3, stride=1, activation_fn=activation, scope='dcn{}/decoder/conv_{}'.format(self.label, 5))
        resnet = tf.contrib.layers.conv2d(resnet, 128, 3, stride=1, activation_fn=None, scope='dcn{}/decoder/conv_{}'.format(self.label, 6))
        inet = tf.add(inet, resnet, name='dcn{}/decoder/sum_c{}'.format(self.label, 2))

        inet = tf.contrib.layers.conv2d(inet, 256, 3, stride=1, activation_fn=activation, scope='dcn{}/decoder/tconv_{}'.format(self.label, 7))
        inet = tf.depth_to_space(inet, 2, name='dcn{}/decoder/d2s_{}'.format(self.label, 7))
        
        inet = tf.contrib.layers.conv2d(inet, 12, 3, stride=1, activation_fn=None, scope='dcn{}/decoder/tconv_{}'.format(self.label, 8))
        inet = tf.depth_to_space(inet, 2, name='dcn{}/decoder/d2s_{}'.format(self.label, 8))
        
        with tf.name_scope('dcn{}/denormalization'.format(self.label)):
            y = (inet + 1) / 2
            
        y = tf.identity(y, name="y")

        with tf.name_scope('dcn{}/optimization'.format(self.label)):
            lr = tf.placeholder(tf.float32, name='dcn_learning_rate')
            loss = tf.nn.l2_loss(self.x - y)
            adam = tf.train.AdamOptimizer(learning_rate=lr)
            opt = adam.minimize(loss, var_list=self.parameters)
            
        return y, lr, loss, adam, opt, latent

In [ ]:
class ResDCN(compression.DCN):
    
    def construct_model(self):
        
        with tf.name_scope('dcn'):

            activation = tf.nn.ResDCN
            last_activation = tf.nn.sigmoid

            print('Building Deep Compression Network with d-latent={}'.format(n_latent))

            net = self.x
            print('net size: {}'.format(net.shape))
        
            # Convolutions
            n_filters = self.n_filters
            
            net = tf.contrib.layers.conv2d(net, 64, self.kernel, stride=2, activation_fn=activation, scope='dcn{}/conv_{}'.format(self.label, 0))
            
            for r in range(self.n_layers):
                net = tf.contrib.layers.conv2d(net, n_filters, self.kernel, stride=2, activation_fn=activation, scope='dcn{}/conv_{}'.format(self.label, r))
            #     print('net size: {}'.format(net.shape))
#                     net = tf.contrib.layers.max_pool2d(net, 2, scope='dcn{}/pool_{}'.format(self.label, r))
                print('net size: {} // {}'.format(net.shape, net))
                n_filters *= self.n_fscale

            # Flatten and get latent representation
            flat = tf.contrib.layers.flatten(net, scope='dcn{}/flat_{}'.format(self.label, 0))
            print('net size: {}'.format(flat.shape))

            latent = tf.contrib.layers.fully_connected(flat, self.n_latent, activation_fn=activation, scope='dcn{}/dense_{}'.format(self.label, 0))
            print('net size: {}'.format(latent.shape))

            inet = tf.contrib.layers.fully_connected(latent, int(flat.shape[-1]), activation_fn=activation, scope='dcn{}/dense_{}'.format(self.label, 1))
            print('net size: {}'.format(inet.shape))
            inet = tf.reshape(net, tf.shape(net), name='dcn{}/reshape_{}'.format(self.label, 0))
            print('net size: {}'.format(inet.shape))

            # Transposed convolutions
            for r in range(self.n_layers):
                inet = tf.contrib.layers.conv2d_transpose(inet, 3 if r == self.n_layers - 1 else n_filters, self.kernel, stride=2, 
                                                          activation_fn=last_activation if r == self.n_layers - 1 else activation,
                                                          scope='dcn{}/tconv_{}'.format(self.label, r))
                print('net size: {}'.format(inet.shape))
                n_filters = n_filters // self.n_fscale

            y = inet

        with tf.name_scope('dcn{}_optimization'.format(self.label)):
            lr = tf.placeholder(tf.float32, name='dcn_learning_rate')
            loss = tf.nn.l2_loss(self.x - y)
            adam = tf.train.AdamOptimizer(learning_rate=lr)
            opt = adam.minimize(loss, var_list=self.parameters)
            
        return y, lr, loss, adam, opt, latent

## Create DCN instance

In [ ]:
graph = tf.Graph()
sess = tf.Session(graph=graph)

dcn = AutoencoderDCN(sess, graph, patch_size=128, n_latent=0, n_layers=4, n_fscale=2, n_filters=16)
# dcn = TwitterDCN(sess, graph, patch_size=128)

print(dcn.summary())
print(dcn.short_name())
# print(dcn.count_parameters_breakdown())
print('Compression stats:', dcn.compression_stats(n_latent_bytes=1/8))

# dcn.load_model('./data/raw/compression/aedcn/8x8x192')
# dcn.save_model(os.path.join('./data/raw/compression/', dcn.short_name()), 3767)

In [ ]:
from helpers import tf_helpers
tf_helpers.show_graph(dcn.graph.as_graph_def())

## Training

In [ ]:
batch_x = data.next_training_batch(batch_id, training['batch_size'], 256)

print(batch_x.shape)
batch_t = np.zeros((batch_x.shape[0], 128, 128, 3), dtype=np.float32)

for i in range(len(batch_x)):
    batch_t[i] = resize(batch_x[i], [patch_size, patch_size], anti_aliasing=True)

print(batch_t.shape)

f = plotting.imsc(batch_x[0:8], ncols=8, figwidth=20)
f = plotting.imsc(batch_t[0:8], ncols=8, figwidth=20)

In [ ]:
dcn.init()

# Compute the number of available batches
n_batches = data['training']['y'].shape[0] // training['batch_size']
v_batches = data['validation']['y'].shape[0] // training['batch_size']

loss = {'training': [], 'validation': []}
loss_ma = deque(maxlen=n_batches)
loss_va = deque(maxlen=v_batches)

# Configure data augmentation
augmentation_probs = {
    'resize': 0.0,
    'flip_h': 0.0,
    'flip_v': 0.0
}

learning_rate = 1e-4
sampling_rate = 100

model_output_dirname = os.path.join('./data/raw/compression/', dcn.short_name())

# Create a summary writer and create the necessary directories
sw = dcn.get_summary_writer(model_output_dirname)

with tqdm.tqdm(total=training['n_epochs'], ncols=120, desc='Train') as pbar:

    for epoch in range(0, training['n_epochs']):
        
        if epoch> 0 and epoch % 1000 == 0:
            learning_rate = learning_rate / 2

        # Iterate through batches of the training data 
        for batch_id in range(n_batches): # TODO n_batches
            
            # Pick random patch size - will be resized later for augmentation
            current_patch = np.random.choice(np.arange(128, 256), 1) if np.random.uniform() < augmentation_probs['resize'] else patch_size
            
            # Sample next batch
            batch_x = data.next_training_batch(batch_id, training['batch_size'], current_patch)
            
            # If rescaling needed, apply
            if patch_size != current_patch:
                batch_t = np.zeros((batch_x.shape[0], patch_size, patch_size, 3), dtype=np.float32)
                for i in range(len(batch_x)):
                    batch_t[i] = resize(batch_x[i], [patch_size, patch_size], anti_aliasing=True)
                batch_x = batch_t                
            
            # Data augmentation - random horizontal flip
            if np.random.uniform() < augmentation_probs['flip_h']: batch_x = batch_x[:, :, ::-1, :]
            if np.random.uniform() < augmentation_probs['flip_v']: batch_x = batch_x[:, ::-1, :, :]
            
            # Make a training step
            loss_value = dcn.training_step(batch_x, learning_rate)
            loss_ma.append(loss_value)

        # Record average values for the whole epoch
        loss['training'].append(np.mean(loss_ma))
            
        # Iterate through batches of the validation data
        if epoch % sampling_rate == 0:
            for batch_id in range(v_batches): # TODO v_batches
                batch_x = data.next_validation_batch(batch_id, training['batch_size'])
                batch_y = dcn.process(batch_x)
                loss_value = np.linalg.norm(batch_x - batch_y)
                loss_va.append(loss_value)
                
            loss['validation'].append(np.mean(loss_va))
            
            # Save current snapshot
            thumbs = (255 * plotting.thumbnails(np.concatenate((batch_x[::2], batch_y[::2]), axis=0), n_cols=8)).astype(np.uint8)
            imageio.imsave(os.path.join(model_output_dirname, 'thumbnails-{:05d}.png'.format(epoch)), thumbs)
            
            # Sample latent space
            batch_z = dcn.compress(batch_x) # data.next_validation_batch(0, 256)
        
            # Save summaries to TB            
            summary = tf.Summary()
            summary.value.add(tag='loss/validation', simple_value=loss['validation'][-1])
            summary.value.add(tag='loss/training', simple_value=loss['training'][-1])        
            summary.value.add(tag='images/reconstructed', image=summaries.log_image(thumbs))
            summary.value.add(tag='histograms/latent', histo=summaries.log_histogram(batch_z))
            sw.add_summary(summary, epoch)
            sw.flush()

        # Update progress bar
        pbar.set_postfix(loss=np.mean(loss['training'][-10:]), loss_v=np.mean(loss['validation'][-1:]), learning_rate='{:.8f}'.format(learning_rate))
        pbar.update(1)

In [ ]:
import matplotlib.pyplot as plt

# batch_z = 

counts, bin_edges = np.histogram(batch_z[1, :], bins=50)

print(batch_z.shape)
print(counts.shape)
print(bin_edges.shape)

print(bin_edges)
print(counts)

plt.bar(bin_edges[:-1], counts)

# plt.hist(batch_z, bins=50)

In [ ]:
batch_x = data.next_validation_batch(0, training['batch_size'])
(batch_id, training['batch_size'])
                
thumbs = (255 * plotting.thumbnails(np.concatenate((batch_x[::2], batch_y[::2]), axis=0), n_cols=8)).astype(np.uint8)
fig = plotting.imsc(thumbs, figwidth=20)

In [ ]:
import matplotlib.pyplot as plt
from helpers import utils

fig = plt.figure(figsize=(20, 4))
ax = fig.gca()
ax.plot(utils.ma_conv(loss['training'], n=11))
# ax.plot(np.arange(0, len(loss['training']), sampling_rate), utils.ma_conv(loss['validation'], n=3))
ax.plot(np.arange(0, len(loss['training']), sampling_rate), loss['validation'], '-o', alpha=0.3)
ax.plot(loss['training'], '.', alpha=0.1)
ax.legend(['train', 'valid'], loc='upper right')
# ax.set_yscale('log')

In [ ]:
# batch_id = (batch_id + 1) % n_batches
batch_x = data.next_training_batch(batch_id, training['batch_size'], patch_size)
fig = plotting.imsc(batch_x[:8], ncols=8, figwidth=20)

In [ ]:
dcn.init()
print(os.listdir(os.path.join('./data/raw/compression/')))
print(os.listdir(os.path.join('./data/raw/compression/', dcn.short_name())))
dcn.load_model(os.path.join('./data/raw/compression/', dcn.short_name()))

In [ ]:
batch_x.max()

In [ ]:
plotting.quickshow()

In [ ]:
# Show a sample and a reconstruction of the current batch
batch_y = dcn.process(batch_x)
f = plotting.imsc(batch_x[0:8], ncols=8, figwidth=20)
f = plotting.imsc(batch_y[0:8], ncols=8, figwidth=20)

In [ ]:
batch_x = data.next_validation_batch(0, 256)

# See latent distribution
batch_z = dcn.compress(batch_x)
print(batch_z.shape)

In [ ]:
from helpers import plotting

# dcn.init()

batch_x = data.next_validation_batch(0, 256)

# See latent distribution
# batch_z = dcn.compress(batch_x).T
batch_z = dcn.compress(batch_x).reshape((256, -1)).T
# batch_z = batch_x.reshape((256, 128*128*3)).T
print(batch_z.shape)

# cov = batch_z.T * batch_z

# cov = np.cov(batch_z)

cov = np.corrcoef(batch_z)

batch_z = batch_z[:9]

fig, axes = plotting.sub(len(batch_z) + 1, ncols=10, figwidth=20)

# print(axes)

for i, ax in enumerate(axes):
   
    if i >= len(batch_z):
        plotting.quickshow(cov)
    else:        
        ax.hist(batch_z[i], bins=30)
        ax.set_yticks([])

In [ ]:
dim_id = 0
fig = plt.figure(figsize=(8, 8))
fig.gca().imshow(batch_x[32,:,:,:])

In [ ]:
dim_id = 0
fig = plt.figure(figsize=(8, 8))
fig.gca().hist(batch_z[dim_id])
print(batch_z[dim_id].shape)

In [ ]:
dcn.save_model('./data/raw/compression/aedcn/8x8x192', epoch)